In [1]:
%load_ext autoreload
%autoreload 2
import sys
import os
print(os.getcwd())
os.chdir('../../complete_project/../')
print(os.getcwd())
# Then set up the paths
# import sys

# import os
# os.environ['PYTHONPATH'] = os.getcwd()  # Now points to thesis_code directory
# sys.path.append(os.environ['PYTHONPATH'])
sys.path.append("/home/caspar/thesis_code/CellOracle")
sys.path.append("/home/caspar/thesis_code/complete_project/py files")
sys.path.append("/home/caspar/thesis_code/complete_project/py files/AIFiles")
sys.path.append("/home/caspar/thesis_code/complete_project/py files/baseGRNConstructionFiles")
sys.path.append("/home/caspar/thesis_code/complete_project/py files/oracleInferenceFiles")
sys.path.append("/home/caspar/thesis_code/complete_project/py files/oracleSetup")
import scanpy as sc
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import anndata
import sys
import logging
from datetime import datetime
import wandb
import optuna
#import modified_celloracle as mco
from CellOracle import celloracle as co

import CellOracleSetup as setup_module
import GRNClusterAnalysis as analysis_module
import GRNInference as inference_module
import GRNInferenceTest as inference_test_module

log_dir = 'logs'
os.makedirs(log_dir, exist_ok=True)
log_filename = os.path.join(log_dir, f"app_{datetime.now().strftime('%Y_%m_%d')}.log")

# Configure the basic logging
logging.basicConfig(
    filename=log_filename,
    filemode='a',
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)

/home/caspar/thesis_code/complete_project/notebooks
/home/caspar/thesis_code


In [2]:
!nvidia-smi


Fri Jun  6 13:09:31 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 555.42.06              Driver Version: 555.42.06      CUDA Version: 12.5     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA RTX 6000 Ada Gene...    Off |   00000000:AC:00.0  On |                  Off |
| 30%   38C    P8             25W /  300W |   14810MiB /  49140MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:

import os
import torch.nn as nn
import AI as ai_module # Import the refactored script
import ParameterTuning as tuning_module # Import the refactored script

data_path_new_data = os.path.join('../celloracle_data', "celloracle_object/new_promoter_without_mescs_trimmed_test_own_umap")
base_config = {
    # Paths
    "ORACLE_PATH": os.path.join(data_path_new_data, "ready_oracle.pkl"),
    "MODEL_SAVE_PATH_BASE": os.path.join("../celloracle_data", "optuna_hpo_models2"), # Base for HPO trial models
    "LOG_DIR_BASE": os.path.join('../celloracle_data', "optuna_hpo_logs2"),         # Base for HPO trial logs

    "HPO_FIXED_MAX_STEPS_PER_EPISODE": 30, 


    "BATCH_SIZE": 128,
    "USE_MASKABLE_PPO": False, 
    "VERBOSE": 0,              
    "LOG_INTERVAL": 10,       
    "RESET_NUM_TIMESTEPS": False,
    "DEVICE": "auto",
    "USE_PROGRESS_BAR": False,  
    "STEP_SAVE_FREQ": 1000000000, 
    "PPO_N_STEPS": 512,
    "PPO_N_EPOCHS": 10,
    "GAMMA": 0.99,
    "GAE_LAMBDA": 0.95,
    "CLIP_RANGE": 0.2,
    "ENT_COEF": 0.01,       
    "VF_COEF": 0.5,
    "MAX_GRAD_NORM": 0.5,
    "LEARNING_RATE": 3e-4,  

    # Environment Specific 
    "ALLOW_GENE_ACTIVATION": True,
    "STANDARD_SD_FACTOR": 1.5,
    "USE_LEARNING_RATE_SCHEDULE": False, 
    "USE_SIMILARITY_REWARD": True,
    "USE_NEW_INPUT": True, 
    "USE_PREV_KNOCKOUT": True, 

    # ENV HYPERPARAMETERS (Some will be tuned by Optuna)
    "DIVIDER_OF_TOTAL_STEPS_FOR_CURRICULUM_LEARNING": 20,
    "TARGET_CELLS_PER_PHASE": 10,
    "MAX_STEP_INCREASE_PER_PHASE": 8,
    "MAX_STEPS_FIRST_PHASE": 50000, 
    "PHASE_STEP_INCREASE": 15000,
    "MAX_STEPS_PER_EPISODE": 36,

    # Other ENV params
    "GENE_ACTIVITY_THRESHOLD": 0.01,
    "TARGET_DISTANCE_THRESHOLD": 0.1,
    "DISTANCE_GAMMA": 0.99,           
    "STEP_PENALTY": -0.05,
    "GOAL_BONUS": 10,
    "FAIL_PENALTY": -3,
    "SAME_CELL_PENALTY": -0.5,
    "DISTANCE_REWARD_SCALE": 5,
    "PARAMETER_TUNING":True,
    "TOTAL_TIMESTEPS_HPO": 320000,


    # NN Architecture (Fixed for this example, but can be tuned)
    "PI_ARCH": [512, 256, 256],
    "VF_ARCH": [512, 256, 256],
    "ACTIVATION_FN": nn.ReLU, # Make sure nn is imported if you use this directly
}

if base_config["USE_SIMILARITY_REWARD"]:
    base_config["DISTANCE_REWARD_SCALE"] = 50
else:
    base_config["DISTANCE_REWARD_SCALE"] = 5

trained_model = tuning_module.run_optuna_hpo(base_config)


[I 2025-06-06 17:44:44,207] A new study created in RDB with name: ppo_celloracle_hpo_20250606_1744



--- Starting Optuna HPO Study: ppo_celloracle_hpo_20250606_1744 ---
test


  0%|          | 0/50 [00:00<?, ?it/s]


---> Starting Optuna Trial 0 <---
  LR: 7.107e-05, ENT_COEF: 0.017, GOAL: 19.5


wandb: ERROR Failed to detect the name of this notebook. You can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: caspar-christiaan (caspar-christiaan-utrecht-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: WARNING Using a boolean value for 'reinit' is deprecated. Use 'return_previous' or 'finish_previous' instead.



--- Starting Training Run ---
Configuration: {'ORACLE_PATH': '../celloracle_data/celloracle_object/new_promoter_without_mescs_trimmed_test_own_umap/ready_oracle.pkl', 'MODEL_SAVE_PATH_BASE': '../celloracle_data/optuna_hpo_models2', 'LOG_DIR_BASE': '../celloracle_data/optuna_hpo_logs2', 'HPO_FIXED_MAX_STEPS_PER_EPISODE': 30, 'BATCH_SIZE': 128, 'USE_MASKABLE_PPO': False, 'VERBOSE': 0, 'LOG_INTERVAL': 10, 'RESET_NUM_TIMESTEPS': False, 'DEVICE': 'auto', 'USE_PROGRESS_BAR': False, 'STEP_SAVE_FREQ': 1000000000, 'PPO_N_STEPS': 256, 'PPO_N_EPOCHS': 7, 'GAMMA': 0.9934533789781249, 'GAE_LAMBDA': 0.95, 'CLIP_RANGE': 0.1116167224336399, 'ENT_COEF': 0.017323522915498706, 'VF_COEF': 0.5202230023486417, 'MAX_GRAD_NORM': 0.5, 'LEARNING_RATE': 7.106591851092232e-05, 'ALLOW_GENE_ACTIVATION': True, 'STANDARD_SD_FACTOR': 1.5, 'USE_LEARNING_RATE_SCHEDULE': False, 'USE_SIMILARITY_REWARD': True, 'USE_NEW_INPUT': True, 'USE_PREV_KNOCKOUT': True, 'DIVIDER_OF_TOTAL_STEPS_FOR_CURRICULUM_LEARNING': 20, 'TARGET_C

wandb: WARNING Found log directory outside of given root_logdir, dropping given root_logdir for event file in ../celloracle_data/optuna_hpo_logs2/trial_0/PPO_0


Agent (PPO) instantiated with architecture: {'pi': [512, 512], 'vf': [512, 512]}
Starting training for 320000 timesteps...
Time taken for simulation:  8.647572040557861
average overall reward:  0.0501329  average fail penalty:  0.0  and average goal bonus:  0.45817143  and average same cell penalty:  -0.005869813 average distance reward:  -0.20607975  and average step penalty:  -0.19608894608379754
Number of done instances:  3
DEBUG (Env): Episode done for env 78. Reward: 20.27, Length: 1
DEBUG (Env): Episode done for env 82. Reward: 21.13, Length: 1
DEBUG (Env): Episode done for env 120. Reward: 19.56, Length: 1
1
Time taken for simulation:  7.315539598464966
average overall reward:  0.118618816  average fail penalty:  0.0  and average goal bonus:  0.45817143  and average same cell penalty:  -0.017609438 average distance reward:  -0.12585427  and average step penalty:  -0.19608894608379754
Number of done instances:  3
DEBUG (Env): Episode done for env 7. Reward: 17.29, Length: 2
DEBUG